In [1]:
import pandas as pd
import numpy as np
from tableone import TableOne
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display
import statsmodels.api as sm
pd.set_option("display.max_columns", None)

/opt/anaconda3/envs/deeplearning/lib/python3.8/site-packages/patsy/constraint.py:13: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.9 it will stop working
  from collections import Mapping


In [2]:
df_orig = pd.read_csv("./data_request.csv")
df_orig.head()

/opt/anaconda3/envs/deeplearning/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,SM_DATE,HPCID,sex,AGE,percentage_fat,Height,Weight,BMI,MVPA,rest_HR,Smoke,CRF,비만도,Muscle_mass,복부지방율,부종검사,Muscle_mass(RA),Muscle_mass(LA),Muscle_mass(BODY),Muscle_mass(RL),Muscle_mass(LL),체지방량,체수분량,제지방량,CRP,CHOLESTEROL,TG,max_heart_rate,BMI_cal,"Glucose, Fasting",ASMI,VO2max,death,delta_time,Diabetes,Hypertension,HTN_med,Hyperlipidemia,Hepatatis,ALC,HDL_C,LDL_C,MBP,SBP,DBP,MED_HYPERTENSION,MED_HYPERLIPIDEMIA,CDW_NO,ID,mean_baPWV,mean_ABI,mean_IMT,AJ_130_Score,Volume_Score
0,2015-08-03,0107034B1E4539,1,55,33.1,153.4,56.6,24.1,0,58.0,0,7.1,115.0,35.8,0.88,0.340,1.94,1.89,17.26,5.53,5.51,18.7,27.9,37.9,0.04,223.0,76.0,119.0,24.052804,90.0,6.319173,24.85,0,1415.0,0,0,0,1,0,0,60.0,156.0,80.0,104.0,64.0,False,False,F7A27BBECBB1,F7A27BBECBB1,1295.0,1.155,0.60,0.0,NaN
1,2012-11-08,01070555203C39,0,29,22.3,169.5,67.8,23.6,0,64.0,0,11.4,107.0,49.8,0.86,0.314,2.86,2.74,23.58,8.40,8.21,15.1,38.5,52.7,NaN,274.0,97.0,175.0,23.598820,97.0,7.730528,39.90,0,2413.0,0,0,0,1,1,1,67.0,197.0,92.2,118.0,75.0,False,0.0,1E9CF1F51D29,NaN,NaN,NaN,NaN,NaN,NaN
2,2012-06-28,0107062321322E,0,36,22.4,176.1,69.4,22.4,0,66.0,1,8.3,102.0,50.9,0.87,0.339,3.09,3.02,24.71,8.45,8.31,15.5,39.7,53.9,1.63,197.0,144.0,156.0,22.379004,87.0,7.374753,29.05,0,2546.0,0,1,0,1,0,0,36.0,142.0,102.8,143.0,76.0,False,0.0,FEDD8A02E7C1,FEDD8A02E7C1,NaN,NaN,0.65,NaN,NaN
3,2012-01-11,010706444B4537,0,50,22.9,171.2,70.5,24.1,1,63.0,0,12.3,109.0,51.4,0.92,0.326,3.00,3.03,24.58,8.33,8.55,16.1,39.9,54.4,0.05,182.0,130.0,158.0,24.053684,86.0,7.816594,43.05,0,2715.0,0,1,0,0,0,0,51.0,111.0,109.4,134.0,93.0,False,0.0,02DA327D3119,02DA327D3119,NaN,NaN,0.55,NaN,NaN
4,2011-06-15,0107064D403C37,0,37,17.9,171.1,71.7,24.5,0,61.0,1,8.2,111.0,55.9,0.88,0.327,3.57,3.48,27.18,9.27,9.08,12.8,43.4,58.9,0.03,205.0,254.0,166.0,24.491712,110.0,8.676283,28.70,0,2925.0,0,1,0,0,0,1,45.0,129.0,108.0,135.0,90.0,False,0.0,3DD843D66B31,3DD843D66B31,NaN,NaN,0.50,NaN,NaN


### Select subset to use

In [3]:
df_orig['Pulse_Pressure'] = df_orig['SBP'] - df_orig['DBP'] ## Pulse Pressure -> Dereived variable
df_orig['Age^2'] = df_orig['AGE'] ** 2
df_orig = df_orig[(df_orig['Hypertension'] != 1) | (df_orig['AJ_130_Score'] >= 0) | (df_orig['Volume_Score'] > 0)] ## Exclude Hypertension 

df_male = df_orig[df_orig['sex'] == False].reset_index(drop=True)
df_female = df_orig[df_orig['sex'] == True].reset_index(drop=True)

/opt/anaconda3/envs/deeplearning/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Make Bins 

In [4]:
bins = [10, 30, 40, 50, 60, 70, 80]
df_male['Age_cat'] = pd.cut(df_male['AGE'], bins=bins)

bins = [20, 30, 40, 50, 60, 70]
df_female['Age_cat'] = pd.cut(df_female['AGE'], bins=bins)

/opt/anaconda3/envs/deeplearning/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [5]:
df_male['CRF_cat_age'] = df_male.groupby('Age_cat')['CRF'].apply(lambda x: pd.qcut(x=x, q=3, labels=['Low', 'Moderate', 'High']))
df_female['CRF_cat_age'] = df_female.groupby('Age_cat')['CRF'].apply(lambda x: pd.qcut(x=x, q=3, labels=['Low', 'Moderate', 'High']))

/opt/anaconda3/envs/deeplearning/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
display(df_male.head())
display(df_female.head())

/opt/anaconda3/envs/deeplearning/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,SM_DATE,HPCID,sex,AGE,percentage_fat,Height,Weight,BMI,MVPA,rest_HR,Smoke,CRF,비만도,Muscle_mass,복부지방율,부종검사,Muscle_mass(RA),Muscle_mass(LA),Muscle_mass(BODY),Muscle_mass(RL),Muscle_mass(LL),체지방량,체수분량,제지방량,CRP,CHOLESTEROL,TG,max_heart_rate,BMI_cal,"Glucose, Fasting",ASMI,VO2max,death,delta_time,Diabetes,Hypertension,HTN_med,Hyperlipidemia,Hepatatis,ALC,HDL_C,LDL_C,MBP,SBP,DBP,MED_HYPERTENSION,MED_HYPERLIPIDEMIA,CDW_NO,ID,mean_baPWV,mean_ABI,mean_IMT,AJ_130_Score,Volume_Score,Pulse_Pressure,Age^2,Age_cat,CRF_cat_age
0,2012-11-08,01070555203C39,0,29,22.3,169.5,67.8,23.6,0,64.0,0,11.4,107.0,49.8,0.86,0.314,2.86,2.74,23.58,8.40,8.21,15.1,38.5,52.7,NaN,274.0,97.0,175.0,23.598820,97.0,7.730528,39.90,0,2413.0,0,0,0,1,1,1,67.0,197.0,92.2,118.0,75.0,False,0.0,1E9CF1F51D29,NaN,NaN,NaN,NaN,NaN,NaN,43.0,841,"(10, 30]",Moderate
1,2013-04-30,010708015F3230,0,52,20.1,163.7,67.3,25.1,1,56.0,0,9.5,114.0,51.1,0.89,0.324,3.32,3.29,25.79,8.02,7.98,13.5,39.6,53.8,0.04,228.0,132.0,147.0,25.114105,94.0,8.437294,33.25,0,2240.0,0,0,0,1,0,1,49.0,156.0,90.0,108.0,78.0,NaN,NaN,370B4C41A959,370B4C41A959,NaN,NaN,0.65,NaN,NaN,30.0,2704,"(50, 60]",Moderate
2,2015-12-04,01070941241234,0,50,28.1,176.1,74.6,24.1,0,62.0,0,9.2,109.0,50.6,0.96,0.338,3.00,2.89,24.30,8.02,8.86,20.9,39.4,53.7,0.03,222.0,132.0,167.0,24.055817,109.0,7.342506,32.20,0,1292.0,0,0,0,1,0,1,56.0,163.0,94.2,120.0,77.0,False,False,F7BFEDDF3119,F7BFEDDF3119,1384.0,1.13,NaN,146.02,96.95,43.0,2500,"(40, 50]",Moderate
3,2012-05-15,01070A15574E38,0,48,22.0,172.0,75.7,25.6,0,58.0,0,8.6,116.0,56.0,0.93,0.328,3.57,3.56,27.53,8.91,8.93,16.6,43.5,59.1,0.08,147.0,150.0,149.0,25.588156,80.0,8.440373,30.10,0,2590.0,0,0,0,0,0,1,51.0,83.0,96.8,125.0,78.0,NaN,NaN,3547640F7DA9,3547640F7DA9,NaN,NaN,0.55,NaN,NaN,47.0,2304,"(40, 50]",Low
4,2014-11-05,01070A15574E38,0,50,22.7,172.2,79.0,26.6,0,54.0,1,9.2,121.0,57.8,0.91,0.330,3.62,3.53,27.56,9.24,9.06,17.9,44.9,61.1,0.11,155.0,125.0,149.0,26.641630,79.0,8.582652,32.20,0,1686.0,0,0,0,0,0,1,61.0,86.0,106.4,134.0,88.0,False,False,3547640F7DA9,3547640F7DA9,NaN,NaN,0.50,NaN,NaN,46.0,2500,"(40, 50]",Moderate


,SM_DATE,HPCID,sex,AGE,percentage_fat,Height,Weight,BMI,MVPA,rest_HR,Smoke,CRF,비만도,Muscle_mass,복부지방율,부종검사,Muscle_mass(RA),Muscle_mass(LA),Muscle_mass(BODY),Muscle_mass(RL),Muscle_mass(LL),체지방량,체수분량,제지방량,CRP,CHOLESTEROL,TG,max_heart_rate,BMI_cal,"Glucose, Fasting",ASMI,VO2max,death,delta_time,Diabetes,Hypertension,HTN_med,Hyperlipidemia,Hepatatis,ALC,HDL_C,LDL_C,MBP,SBP,DBP,MED_HYPERTENSION,MED_HYPERLIPIDEMIA,CDW_NO,ID,mean_baPWV,mean_ABI,mean_IMT,AJ_130_Score,Volume_Score,Pulse_Pressure,Age^2,Age_cat,CRF_cat_age
0,2015-08-03,0107034B1E4539,1,55,33.1,153.4,56.6,24.1,0,58.0,0,7.1,115.0,35.8,0.88,0.340,1.94,1.89,17.26,5.53,5.51,18.7,27.9,37.9,0.04,223.0,76.0,119.0,24.052804,90.0,6.319173,24.85,0,1415.0,0,0,0,1,0,0,60.0,156.0,80.0,104.0,64.0,False,False,F7A27BBECBB1,F7A27BBECBB1,1295.0,1.155,0.60,0.0,NaN,40.0,3025,"(50, 60]",Moderate
1,2013-12-24,01070F0052433A,1,51,34.3,157.1,65.5,26.5,0,59.0,0,7.3,126.0,40.5,0.85,0.337,2.07,2.10,18.58,6.59,6.55,22.5,31.6,43.0,0.04,231.0,88.0,136.0,26.539267,86.0,7.013660,25.55,0,2002.0,0,0,0,1,0,0,71.0,151.0,71.4,96.0,55.0,NaN,NaN,11B7C3D709D9,11B7C3D709D9,NaN,NaN,0.95,NaN,NaN,41.0,2601,"(50, 60]",Moderate
2,2014-06-20,01071C584B594B,1,57,22.5,163.2,53.5,20.1,0,69.0,0,6.1,96.0,39.2,0.84,0.337,2.02,1.94,18.16,6.53,6.50,12.0,30.5,41.5,0.03,234.0,64.0,163.0,20.086926,86.0,6.379007,21.35,0,1824.0,0,0,0,1,0,1,81.0,150.0,73.8,90.0,63.0,False,False,A29A329B0569,A29A329B0569,1387.5,1.240,NaN,NaN,NaN,27.0,3249,"(50, 60]",Low
3,2013-03-25,01072340176112,1,55,36.0,153.2,60.6,25.8,0,55.0,0,6.6,123.0,36.6,0.87,0.335,1.92,1.95,17.52,5.72,5.76,21.8,28.5,38.8,0.03,192.0,97.0,133.0,25.819932,94.0,6.540197,23.10,0,2276.0,0,0,0,0,0,0,81.0,98.0,83.8,106.0,69.0,False,0.0,373BF2696159,373BF2696159,1164.5,1.165,0.55,NaN,NaN,37.0,3025,"(50, 60]",Low
4,2012-09-04,01072D3E02090F,1,46,24.3,152.4,49.4,21.3,1,58.0,0,9.8,101.0,35.4,0.80,0.339,1.81,1.82,16.60,5.83,5.67,12.0,27.6,37.4,0.03,171.0,47.0,150.0,21.269487,80.0,6.514319,34.30,0,2478.0,0,0,0,0,0,1,72.0,99.0,70.2,93.0,55.0,False,0.0,76528DD3E159,NaN,NaN,NaN,NaN,NaN,NaN,38.0,2116,"(40, 50]",High


## Table 1

In [7]:
columns = ['AGE', 'Height', 'Weight', 'BMI', 'percentage_fat', 'SBP', 'DBP', 'Pulse_Pressure', 
           'mean_baPWV', 'mean_ABI', 'mean_IMT', 'AJ_130_Score', 'Volume_Score', 'CRF', 
           'rest_HR', 'max_heart_rate', 'Glucose, Fasting', 'TG', 'CHOLESTEROL', 'Smoke', 'ALC', 'MVPA', 
           'Diabetes', 'Hyperlipidemia']


categorical = ['MVPA', 'Smoke', 'ALC', 'Diabetes', 'Hyperlipidemia']

dem_table = TableOne(data=df_orig, columns=columns, categorical=categorical, groupby='sex', pval=True)

/opt/anaconda3/envs/deeplearning/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [8]:
display(dem_table)
dem_table.to_excel("./table_1.xlsx")

/opt/anaconda3/envs/deeplearning/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Grouped by sex                                                        
                                     Missing         Overall               0               1 P-Value
n                                                      10761            9200            1561        
AGE, mean (SD)                             0      50.0 (7.0)      49.5 (7.1)      52.6 (5.8)  <0.001
Height, mean (SD)                          0     170.2 (7.0)     172.0 (5.5)     159.5 (5.0)  <0.001
Weight, mean (SD)                          0     70.1 (10.3)      72.5 (8.8)      56.3 (6.5)  <0.001
BMI, mean (SD)                             0      24.1 (2.6)      24.5 (2.5)      22.1 (2.4)  <0.001
percentage_fat, mean (SD)                  0      23.5 (5.3)      22.5 (4.6)      29.3 (5.3)  <0.001
SBP, mean (SD)                             8    116.3 (12.5)    117.3 (12.3)    110.3 (12.4)  <0.001
DBP, mean (SD)                             8      74.7 (9.2)      76.1 (8.4)      66.5 (9.4)  <0.001
Pulse_Pressure, mean (SD)                  8      41.5 (9.3)      41.1 (9.1)     43.9 (10.2)  <0.001
mean_baPWV, mean (SD)                   6804  1353.9 (172.3)  1359.8 (175.1)  1332.1 (159.6)  <0.001
mean_ABI, mean (SD)                     6804       1.1 (0.1)       1.1 (0.1)       1.1 (0.2)   0.859
mean_IMT, mean (SD)                     2774       0.6 (0.1)       0.6 (0.1)       0.6 (0.1)  <0.001
AJ_130_Score, mean (SD)                 7345    42.3 (126.5)    43.7 (129.5)     18.2 (43.2)  <0.001
Volume_Score, mean (SD)                 9261    79.0 (138.4)    80.7 (140.6)     37.2 (38.9)  <0.001
CRF, mean (SD)                             0       9.1 (1.3)       9.4 (1.1)       7.4 (0.9)  <0.001
rest_HR, mean (SD)                         0      62.4 (8.0)      62.5 (8.1)      61.7 (7.4)  <0.001
max_heart_rate, mean (SD)                  0    156.9 (23.9)    158.1 (20.0)    149.9 (38.8)  <0.001
Glucose, Fasting, mean (SD)                8     97.9 (18.4)     98.9 (19.1)     92.0 (12.4)  <0.001
TG, mean (SD)                            180    136.3 (86.9)    142.1 (90.2)    101.3 (50.7)  <0.001
CHOLESTEROL, mean (SD)                   180    199.0 (34.4)    198.6 (34.7)    201.3 (32.7)   0.002
Smoke, n (%)                0              0     7039 (65.4)     5521 (60.0)     1518 (97.2)  <0.001
                            1                    3722 (34.6)     3679 (40.0)        43 (2.8)        
ALC, n (%)                  0              0     2301 (21.4)     1393 (15.1)      908 (58.2)  <0.001
                            1                    8460 (78.6)     7807 (84.9)      653 (41.8)        
MVPA, n (%)                 0              0     9387 (87.2)     7977 (86.7)     1410 (90.3)  <0.001
                            1                    1374 (12.8)     1223 (13.3)       151 (9.7)        
Diabetes, n (%)             0              0     9820 (91.3)     8317 (90.4)     1503 (96.3)  <0.001
                            1                      941 (8.7)       883 (9.6)        58 (3.7)        
Hyperlipidemia, n (%)       0              0     4515 (42.0)     3711 (40.3)      804 (51.5)  <0.001
                            1                    6246 (58.0)     5489 (59.7)      757 (48.5)

## Table 2

In [9]:
columns = ['AGE', 'Height', 'Weight', 'BMI', 'percentage_fat', 'SBP', 'DBP', 'Pulse_Pressure', 
           'mean_baPWV', 'mean_ABI', 'mean_IMT', 'AJ_130_Score', 'Volume_Score', 'CRF', 
           'rest_HR', 'max_heart_rate', 'Glucose, Fasting', 'TG', 'CHOLESTEROL', 'Smoke', 'ALC', 'MVPA', 
           'Diabetes', 'Hyperlipidemia']


categorical = ['MVPA', 'Smoke', 'ALC', 'Diabetes', 'Hyperlipidemia']

group_by = 'CRF_cat_age'

dem_table = TableOne(data=df_male, columns=columns, categorical=categorical, groupby=group_by, pval=True)

display(dem_table)
dem_table.to_excel("./table_2_1.xlsx")

/opt/anaconda3/envs/deeplearning/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/opt/anaconda3/envs/deeplearning/lib/python3.8/site-packages/tableone/tableone.py:1420: UserWarning: Order variable not found: Age_cat
  warnings.warn("Order variable not found: {}".format(k))


Grouped by CRF_cat_age                                                                        
                                             Missing         Overall             Low        Moderate            High P-Value
n                                                               9200            3224            3039            2937        
AGE, mean (SD)                                     0      49.5 (7.1)      50.0 (7.1)      49.5 (7.1)      49.0 (7.1)  <0.001
Height, mean (SD)                                  0     172.0 (5.5)     172.2 (5.6)     172.1 (5.6)     171.6 (5.4)  <0.001
Weight, mean (SD)                                  0      72.5 (8.8)      74.6 (9.8)      72.1 (8.3)      70.6 (7.6)  <0.001
BMI, mean (SD)                                     0      24.5 (2.5)      25.1 (2.9)      24.3 (2.4)      24.0 (2.1)  <0.001
percentage_fat, mean (SD)                          0      22.5 (4.6)      24.4 (4.7)      22.3 (4.3)      20.6 (4.1)  <0.001
SBP, mean (SD)                                     7    117.3 (12.3)    117.5 (12.9)    116.9 (11.8)    117.3 (12.1)   0.148
DBP, mean (SD)                                     7      76.1 (8.4)      76.6 (8.6)      76.2 (8.3)      75.6 (8.4)  <0.001
Pulse_Pressure, mean (SD)                          7      41.1 (9.1)      41.0 (9.4)      40.8 (8.8)      41.7 (9.1)   0.001
mean_baPWV, mean (SD)                           6081  1359.8 (175.1)  1379.6 (161.6)  1355.9 (148.7)  1341.1 (212.5)  <0.001
mean_ABI, mean (SD)                             6081       1.1 (0.1)       1.1 (0.1)       1.2 (0.1)       1.2 (0.1)   0.004
mean_IMT, mean (SD)                             1788       0.6 (0.1)       0.6 (0.1)       0.6 (0.1)       0.6 (0.1)  <0.001
AJ_130_Score, mean (SD)                         5969    43.7 (129.5)    57.9 (162.4)    36.3 (103.9)    34.6 (106.4)  <0.001
Volume_Score, mean (SD)                         7756    80.7 (140.6)    94.6 (167.7)    74.5 (112.8)    67.0 (120.8)   0.005
CRF, mean (SD)                                     0       9.4 (1.1)       8.3 (0.6)       9.4 (0.5)      10.6 (0.8)  <0.001
rest_HR, mean (SD)                                 0      62.5 (8.1)      64.7 (8.4)      62.2 (7.8)      60.2 (7.5)  <0.001
max_heart_rate, mean (SD)                          0    158.1 (20.0)    156.2 (21.0)    158.5 (11.6)    159.9 (25.0)  <0.001
Glucose, Fasting, mean (SD)                        7     98.9 (19.1)    100.3 (21.1)     98.9 (19.2)     97.6 (16.3)  <0.001
TG, mean (SD)                                    124    142.1 (90.2)    156.4 (95.9)    142.3 (84.1)    126.3 (87.2)  <0.001
CHOLESTEROL, mean (SD)                           124    198.6 (34.7)    200.0 (36.3)    198.0 (34.0)    197.6 (33.5)   0.014
Smoke, n (%)                0                      0     5521 (60.0)     1868 (57.9)     1779 (58.5)     1874 (63.8)  <0.001
                            1                            3679 (40.0)     1356 (42.1)     1260 (41.5)     1063 (36.2)        
ALC, n (%)                  0                      0     1393 (15.1)      596 (18.5)      417 (13.7)      380 (12.9)  <0.001
                            1                            7807 (84.9)     2628 (81.5)     2622 (86.3)     2557 (87.1)        
MVPA, n (%)                 0                      0     7977 (86.7)     3012 (93.4)     2698 (88.8)     2267 (77.2)  <0.001
                            1                            1223 (13.3)       212 (6.6)      341 (11.2)      670 (22.8)        
Diabetes, n (%)             0                      0     8317 (90.4)     2848 (88.3)     2766 (91.0)     2703 (92.0)  <0.001
                            1                              883 (9.6)      376 (11.7)       273 (9.0)       234 (8.0)        
Hyperlipidemia, n (%)       0                      0     3711 (40.3)     1096 (34.0)     1274 (41.9)     1341 (45.7)  <0.001
                            1                            5489 (59.7)     2128 (66.0)     1765 (58.1)     1596 (54.3)

In [10]:
columns = ['AGE', 'Height', 'Weight', 'BMI', 'percentage_fat', 'SBP', 'DBP', 'Pulse_Pressure', 
           'mean_baPWV', 'mean_ABI', 'mean_IMT', 'AJ_130_Score', 'Volume_Score', 'CRF', 
           'rest_HR', 'max_heart_rate', 'Glucose, Fasting', 'TG', 'CHOLESTEROL', 'Smoke', 'ALC', 'MVPA', 
           'Diabetes', 'Hyperlipidemia']


categorical = ['MVPA', 'Smoke', 'ALC', 'Diabetes', 'Hyperlipidemia']

group_by = 'CRF_cat_age'

dem_table = TableOne(data=df_female, columns=columns, categorical=categorical, groupby=group_by, pval=True, pval_test_name=True)

display(dem_table)
dem_table.to_excel("./table_2_2.xlsx")

/opt/anaconda3/envs/deeplearning/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/opt/anaconda3/envs/deeplearning/lib/python3.8/site-packages/tableone/tableone.py:236: DeprecationWarning: The pval_test_name argument is deprecated; use htest_name instead.
  warnings.warn("The pval_test_name argument is deprecated; use "
/opt/anaconda3/envs/deeplearning/lib/python3.8/site-packages/tableone/tableone.py:1420: UserWarning: Order variable not found: Age_cat
  warnings.warn("Order variable not found: {}".format(k))


Grouped by CRF_cat_age                                                                                       
                                             Missing         Overall             Low        Moderate            High P-Value           Test
n                                                               1561             554             507             500                       
AGE, mean (SD)                                     0      52.6 (5.8)      52.7 (5.8)      52.7 (5.6)      52.2 (6.1)   0.338  One-way ANOVA
Height, mean (SD)                                  0     159.5 (5.0)     159.5 (5.1)     159.5 (4.9)     159.4 (4.9)   0.930  One-way ANOVA
Weight, mean (SD)                                  0      56.3 (6.5)      57.5 (6.9)      56.3 (6.3)      54.8 (5.8)  <0.001  One-way ANOVA
BMI, mean (SD)                                     0      22.1 (2.4)      22.6 (2.5)      22.1 (2.3)      21.6 (2.1)  <0.001  One-way ANOVA
percentage_fat, mean (SD)                          0      29.3 (5.3)      31.1 (5.0)      29.4 (5.1)      27.4 (5.1)  <0.001  One-way ANOVA
SBP, mean (SD)                                     1    110.3 (12.4)    110.8 (13.0)    110.9 (12.0)    109.2 (11.9)   0.048  One-way ANOVA
DBP, mean (SD)                                     1      66.5 (9.4)      66.7 (9.8)      66.7 (8.9)      65.9 (9.6)   0.283  One-way ANOVA
Pulse_Pressure, mean (SD)                          1     43.9 (10.2)     44.1 (10.6)     44.2 (10.2)      43.3 (9.6)   0.313  One-way ANOVA
mean_baPWV, mean (SD)                            723  1332.1 (159.6)  1339.1 (173.3)  1335.4 (140.2)  1320.5 (164.4)   0.363  One-way ANOVA
mean_ABI, mean (SD)                              723       1.1 (0.2)       1.1 (0.1)       1.2 (0.4)       1.2 (0.1)   0.204  One-way ANOVA
mean_IMT, mean (SD)                              986       0.6 (0.1)       0.6 (0.1)       0.6 (0.1)       0.6 (0.1)   0.349  One-way ANOVA
AJ_130_Score, mean (SD)                         1376     18.2 (43.2)     28.8 (54.4)      9.3 (24.4)     17.3 (44.5)   0.050  One-way ANOVA
Volume_Score, mean (SD)                         1505     37.2 (38.9)     52.1 (37.0)     21.3 (26.8)     37.2 (44.8)   0.061  One-way ANOVA
CRF, mean (SD)                                     0       7.4 (0.9)       6.6 (0.5)       7.4 (0.3)       8.4 (0.5)  <0.001  One-way ANOVA
rest_HR, mean (SD)                                 0      61.7 (7.4)      63.2 (7.3)      61.9 (7.1)      60.0 (7.6)  <0.001  One-way ANOVA
max_heart_rate, mean (SD)                          0    149.9 (38.8)    147.3 (62.1)    150.5 (14.6)    152.2 (14.0)   0.112  One-way ANOVA
Glucose, Fasting, mean (SD)                        1     92.0 (12.4)     92.6 (12.3)     92.3 (13.8)     90.9 (10.8)   0.061  One-way ANOVA
TG, mean (SD)                                     56    101.3 (50.7)    104.9 (49.9)    102.9 (50.5)     95.7 (51.5)   0.011  One-way ANOVA
CHOLESTEROL, mean (SD)                            56    201.3 (32.7)    200.9 (32.1)    201.1 (32.9)    202.0 (33.3)   0.855  One-way ANOVA
Smoke, n (%)                0                      0     1518 (97.2)      539 (97.3)      492 (97.0)      487 (97.4)   0.938    Chi-squared
                            1                               43 (2.8)        15 (2.7)        15 (3.0)        13 (2.6)                       
ALC, n (%)                  0                      0      908 (58.2)      335 (60.5)      309 (60.9)      264 (52.8)   0.013    Chi-squared
                            1                             653 (41.8)      219 (39.5)      198 (39.1)      236 (47.2)                       
MVPA, n (%)                 0                      0     1410 (90.3)      516 (93.1)      469 (92.5)      425 (85.0)  <0.001    Chi-squared
                            1                              151 (9.7)        38 (6.9)        38 (7.5)       75 (15.0)                       
Diabetes, n (%)             0                      0     1503 (96.3)      538 (97.1)      477 (94.1)      488 

## Table 3

### Linear Regression Analysis

##### Outcome 
* baPWV
* CAROTID IMT
* ABI
* AJ-130 Score
* Volume Score
* SBP
* DBP
* Pulse Pressure

### Model 1

In [11]:
feature_mask = ['AGE', 'Age^2'] # Feature
feature_mask = ['AGE']

/opt/anaconda3/envs/deeplearning/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


#### Male

In [12]:
df_male_exog = sm.add_constant(df_male[df_male['mean_baPWV'].notnull()][feature_mask], prepend=False)
outcome = df_male[df_male['mean_baPWV'].notnull()]['mean_baPWV']

mod = sm.OLS(outcome.astype(float), df_male_exog.astype(float))
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:             mean_baPWV   R-squared:                       0.058
Model:                            OLS   Adj. R-squared:                  0.057
Method:                 Least Squares   F-statistic:                     190.2
Date:                Fri, 25 Jun 2021   Prob (F-statistic):           4.87e-42
Time:                        01:14:21   Log-Likelihood:                -20443.
No. Observations:                3119   AIC:                         4.089e+04
Df Residuals:                    3117   BIC:                         4.090e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
AGE            6.4486      0.468     13.790      0.0

/opt/anaconda3/envs/deeplearning/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [13]:
df_male_exog = sm.add_constant(df_male[df_male['mean_ABI'].notnull()][feature_mask], prepend=False)
outcome = df_male[df_male['mean_ABI'].notnull()]['mean_ABI']

mod = sm.OLS(outcome.astype(float), df_male_exog.astype(float))
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:               mean_ABI   R-squared:                       0.035
Model:                            OLS   Adj. R-squared:                  0.035
Method:                 Least Squares   F-statistic:                     113.2
Date:                Fri, 25 Jun 2021   Prob (F-statistic):           5.30e-26
Time:                        01:14:21   Log-Likelihood:                 4333.4
No. Observations:                3119   AIC:                            -8663.
Df Residuals:                    3117   BIC:                            -8651.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
AGE            0.0018      0.000     10.642      0.0

/opt/anaconda3/envs/deeplearning/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [14]:
df_male_exog = sm.add_constant(df_male[df_male['mean_IMT'].notnull()][feature_mask], prepend=False)
outcome = df_male[df_male['mean_IMT'].notnull()]['mean_IMT']

mod = sm.OLS(outcome.astype(float), df_male_exog.astype(float))
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:               mean_IMT   R-squared:                       0.141
Model:                            OLS   Adj. R-squared:                  0.141
Method:                 Least Squares   F-statistic:                     1215.
Date:                Fri, 25 Jun 2021   Prob (F-statistic):          1.41e-246
Time:                        01:14:21   Log-Likelihood:                 4274.0
No. Observations:                7412   AIC:                            -8544.
Df Residuals:                    7410   BIC:                            -8530.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
AGE            0.0081      0.000     34.852      0.0

/opt/anaconda3/envs/deeplearning/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [15]:
df_male_exog = sm.add_constant(df_male[df_male['AJ_130_Score'].notnull()][feature_mask], prepend=False)
outcome = df_male[df_male['AJ_130_Score'].notnull()]['AJ_130_Score']

mod = sm.OLS(outcome.astype(float), df_male_exog.astype(float))
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:           AJ_130_Score   R-squared:                       0.050
Model:                            OLS   Adj. R-squared:                  0.049
Method:                 Least Squares   F-statistic:                     168.5
Date:                Fri, 25 Jun 2021   Prob (F-statistic):           1.37e-37
Time:                        01:14:21   Log-Likelihood:                -20217.
No. Observations:                3231   AIC:                         4.044e+04
Df Residuals:                    3229   BIC:                         4.045e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
AGE            5.2677      0.406     12.980      0.0

/opt/anaconda3/envs/deeplearning/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [16]:
df_male_exog = sm.add_constant(df_male[df_male['Volume_Score'].notnull()][feature_mask], prepend=False)
outcome = df_male[df_male['Volume_Score'].notnull()]['Volume_Score']

mod = sm.OLS(outcome.astype(float), df_male_exog.astype(float))
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:           Volume_Score   R-squared:                       0.036
Model:                            OLS   Adj. R-squared:                  0.036
Method:                 Least Squares   F-statistic:                     54.20
Date:                Fri, 25 Jun 2021   Prob (F-statistic):           3.03e-13
Time:                        01:14:21   Log-Likelihood:                -9163.3
No. Observations:                1444   AIC:                         1.833e+04
Df Residuals:                    1442   BIC:                         1.834e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
AGE            5.0325      0.684      7.362      0.0

/opt/anaconda3/envs/deeplearning/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [17]:
df_male_exog = sm.add_constant(df_male[df_male['SBP'].notnull()][feature_mask], prepend=False)
outcome = df_male[df_male['SBP'].notnull()]['SBP']

mod = sm.OLS(outcome.astype(float), df_male_exog.astype(float))
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                    SBP   R-squared:                       0.008
Model:                            OLS   Adj. R-squared:                  0.008
Method:                 Least Squares   F-statistic:                     71.65
Date:                Fri, 25 Jun 2021   Prob (F-statistic):           2.97e-17
Time:                        01:14:21   Log-Likelihood:                -36070.
No. Observations:                9193   AIC:                         7.214e+04
Df Residuals:                    9191   BIC:                         7.216e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
AGE           -0.1515      0.018     -8.464      0.0

/opt/anaconda3/envs/deeplearning/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [18]:
df_male_exog = sm.add_constant(df_male[df_male['DBP'].notnull()][feature_mask], prepend=False)
outcome = df_male[df_male['DBP'].notnull()]['DBP']

mod = sm.OLS(outcome.astype(float), df_male_exog.astype(float))
res = mod.fit()
print(res.summary())

/opt/anaconda3/envs/deeplearning/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


                            OLS Regression Results                            
Dep. Variable:                    DBP   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                   0.04024
Date:                Fri, 25 Jun 2021   Prob (F-statistic):              0.841
Time:                        01:14:21   Log-Likelihood:                -32634.
No. Observations:                9193   AIC:                         6.527e+04
Df Residuals:                    9191   BIC:                         6.529e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
AGE           -0.0025      0.012     -0.201      0.8

In [19]:
df_male_exog = sm.add_constant(df_male[df_male['Pulse_Pressure'].notnull()][feature_mask], prepend=False)
outcome = df_male[df_male['Pulse_Pressure'].notnull()]['Pulse_Pressure']

mod = sm.OLS(outcome.astype(float), df_male_exog.astype(float))
res = mod.fit()
print(res.summary())

/opt/anaconda3/envs/deeplearning/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


                            OLS Regression Results                            
Dep. Variable:         Pulse_Pressure   R-squared:                       0.014
Model:                            OLS   Adj. R-squared:                  0.014
Method:                 Least Squares   F-statistic:                     127.3
Date:                Fri, 25 Jun 2021   Prob (F-statistic):           2.51e-29
Time:                        01:14:21   Log-Likelihood:                -33277.
No. Observations:                9193   AIC:                         6.656e+04
Df Residuals:                    9191   BIC:                         6.657e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
AGE           -0.1490      0.013    -11.282      0.0

### Female

In [20]:
df_data = df_female

df_data_exog = sm.add_constant(df_data[df_data['mean_baPWV'].notnull()][feature_mask], prepend=False)
outcome = df_data[df_data['mean_baPWV'].notnull()]['mean_baPWV']

mod = sm.OLS(outcome.astype(float), df_data_exog.astype(float))
res = mod.fit()
print(res.summary())

/opt/anaconda3/envs/deeplearning/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


                            OLS Regression Results                            
Dep. Variable:             mean_baPWV   R-squared:                       0.134
Model:                            OLS   Adj. R-squared:                  0.133
Method:                 Least Squares   F-statistic:                     129.8
Date:                Fri, 25 Jun 2021   Prob (F-statistic):           4.65e-28
Time:                        01:14:21   Log-Likelihood:                -5379.1
No. Observations:                 838   AIC:                         1.076e+04
Df Residuals:                     836   BIC:                         1.077e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
AGE           11.8275      1.038     11.394      0.0

In [21]:
df_data = df_female

df_data_exog = sm.add_constant(df_data[df_data['mean_ABI'].notnull()][feature_mask], prepend=False)
outcome = df_data[df_data['mean_ABI'].notnull()]['mean_ABI']

mod = sm.OLS(outcome.astype(float), df_data_exog.astype(float))
res = mod.fit()
print(res.summary())

/opt/anaconda3/envs/deeplearning/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


                            OLS Regression Results                            
Dep. Variable:               mean_ABI   R-squared:                       0.012
Model:                            OLS   Adj. R-squared:                  0.011
Method:                 Least Squares   F-statistic:                     10.49
Date:                Fri, 25 Jun 2021   Prob (F-statistic):            0.00125
Time:                        01:14:21   Log-Likelihood:                 41.842
No. Observations:                 838   AIC:                            -79.68
Df Residuals:                     836   BIC:                            -70.22
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
AGE            0.0052      0.002      3.239      0.0

In [22]:
df_data = df_female

df_data_exog = sm.add_constant(df_data[df_data['mean_IMT'].notnull()][feature_mask], prepend=False)
outcome = df_data[df_data['mean_IMT'].notnull()]['mean_IMT']

mod = sm.OLS(outcome.astype(float), df_data_exog.astype(float))
res = mod.fit()
print(res.summary())

/opt/anaconda3/envs/deeplearning/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


                            OLS Regression Results                            
Dep. Variable:               mean_IMT   R-squared:                       0.170
Model:                            OLS   Adj. R-squared:                  0.169
Method:                 Least Squares   F-statistic:                     117.6
Date:                Fri, 25 Jun 2021   Prob (F-statistic):           4.82e-25
Time:                        01:14:21   Log-Likelihood:                 448.31
No. Observations:                 575   AIC:                            -892.6
Df Residuals:                     573   BIC:                            -883.9
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
AGE            0.0081      0.001     10.843      0.0

In [23]:
df_data = df_female

df_data_exog = sm.add_constant(df_data[df_data['AJ_130_Score'].notnull()][feature_mask], prepend=False)
outcome = df_data[df_data['AJ_130_Score'].notnull()]['AJ_130_Score']

mod = sm.OLS(outcome.astype(float), df_data_exog.astype(float))
res = mod.fit()
print(res.summary())

/opt/anaconda3/envs/deeplearning/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


                            OLS Regression Results                            
Dep. Variable:           AJ_130_Score   R-squared:                       0.091
Model:                            OLS   Adj. R-squared:                  0.086
Method:                 Least Squares   F-statistic:                     18.39
Date:                Fri, 25 Jun 2021   Prob (F-statistic):           2.91e-05
Time:                        01:14:21   Log-Likelihood:                -949.88
No. Observations:                 185   AIC:                             1904.
Df Residuals:                     183   BIC:                             1910.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
AGE            2.4887      0.580      4.288      0.0

In [24]:
df_data = df_female

df_data_exog = sm.add_constant(df_data[df_data['Volume_Score'].notnull()][feature_mask], prepend=False)
outcome = df_data[df_data['Volume_Score'].notnull()]['Volume_Score']

mod = sm.OLS(outcome.astype(float), df_data_exog.astype(float))
res = mod.fit()
print(res.summary())

/opt/anaconda3/envs/deeplearning/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


                            OLS Regression Results                            
Dep. Variable:           Volume_Score   R-squared:                       0.037
Model:                            OLS   Adj. R-squared:                  0.020
Method:                 Least Squares   F-statistic:                     2.103
Date:                Fri, 25 Jun 2021   Prob (F-statistic):              0.153
Time:                        01:14:21   Log-Likelihood:                -282.88
No. Observations:                  56   AIC:                             569.8
Df Residuals:                      54   BIC:                             573.8
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
AGE            1.9742      1.361      1.450      0.1

In [25]:
df_data = df_female

df_data_exog = sm.add_constant(df_data[df_data['SBP'].notnull()][feature_mask], prepend=False)
outcome = df_data[df_data['SBP'].notnull()]['SBP']

mod = sm.OLS(outcome.astype(float), df_data_exog.astype(float))
res = mod.fit()
print(res.summary())

/opt/anaconda3/envs/deeplearning/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


                            OLS Regression Results                            
Dep. Variable:                    SBP   R-squared:                       0.013
Model:                            OLS   Adj. R-squared:                  0.013
Method:                 Least Squares   F-statistic:                     21.31
Date:                Fri, 25 Jun 2021   Prob (F-statistic):           4.23e-06
Time:                        01:14:21   Log-Likelihood:                -6126.1
No. Observations:                1560   AIC:                         1.226e+04
Df Residuals:                    1558   BIC:                         1.227e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
AGE            0.2466      0.053      4.616      0.0

In [26]:
df_data = df_female

df_data_exog = sm.add_constant(df_data[df_data['DBP'].notnull()][feature_mask], prepend=False)
outcome = df_data[df_data['DBP'].notnull()]['DBP']

mod = sm.OLS(outcome.astype(float), df_data_exog.astype(float))
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                    DBP   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     1.687
Date:                Fri, 25 Jun 2021   Prob (F-statistic):              0.194
Time:                        01:14:21   Log-Likelihood:                -5715.3
No. Observations:                1560   AIC:                         1.143e+04
Df Residuals:                    1558   BIC:                         1.145e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
AGE            0.0533      0.041      1.299      0.1

/opt/anaconda3/envs/deeplearning/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Model 2

In [27]:
feature_mask = ['AGE', 'Age^2', 'percentage_fat'] # Feature
feature_mask = ['AGE', 'percentage_fat']

/opt/anaconda3/envs/deeplearning/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


#### Male

In [28]:
df_data = df_male

df_data_exog = sm.add_constant(df_data[df_data['mean_baPWV'].notnull()][feature_mask], prepend=False)
outcome = df_data[df_data['mean_baPWV'].notnull()]['mean_baPWV']

mod = sm.OLS(outcome.astype(float), df_data_exog.astype(float))
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:             mean_baPWV   R-squared:                       0.062
Model:                            OLS   Adj. R-squared:                  0.061
Method:                 Least Squares   F-statistic:                     102.1
Date:                Fri, 25 Jun 2021   Prob (F-statistic):           1.10e-43
Time:                        01:14:21   Log-Likelihood:                -20437.
No. Observations:                3119   AIC:                         4.088e+04
Df Residuals:                    3116   BIC:                         4.090e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
AGE                6.3521      0.467     13.

/opt/anaconda3/envs/deeplearning/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [29]:
df_data = df_male

df_data_exog = sm.add_constant(df_data[df_data['mean_ABI'].notnull()][feature_mask], prepend=False)
outcome = df_data[df_data['mean_ABI'].notnull()]['mean_ABI']

mod = sm.OLS(outcome.astype(float), df_data_exog.astype(float))
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:               mean_ABI   R-squared:                       0.035
Model:                            OLS   Adj. R-squared:                  0.034
Method:                 Least Squares   F-statistic:                     56.62
Date:                Fri, 25 Jun 2021   Prob (F-statistic):           7.04e-25
Time:                        01:14:21   Log-Likelihood:                 4333.4
No. Observations:                3119   AIC:                            -8661.
Df Residuals:                    3116   BIC:                            -8643.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
AGE                0.0018      0.000     10.

/opt/anaconda3/envs/deeplearning/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [30]:
df_data = df_male

df_data_exog = sm.add_constant(df_data[df_data['mean_IMT'].notnull()][feature_mask], prepend=False)
outcome = df_data[df_data['mean_IMT'].notnull()]['mean_IMT']

mod = sm.OLS(outcome.astype(float), df_data_exog.astype(float))
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:               mean_IMT   R-squared:                       0.147
Model:                            OLS   Adj. R-squared:                  0.147
Method:                 Least Squares   F-statistic:                     640.5
Date:                Fri, 25 Jun 2021   Prob (F-statistic):          2.66e-257
Time:                        01:14:21   Log-Likelihood:                 4302.4
No. Observations:                7412   AIC:                            -8599.
Df Residuals:                    7409   BIC:                            -8578.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
AGE                0.0080      0.000     34.

/opt/anaconda3/envs/deeplearning/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [31]:
df_data = df_male

df_data_exog = sm.add_constant(df_data[df_data['AJ_130_Score'].notnull()][feature_mask], prepend=False)
outcome = df_data[df_data['AJ_130_Score'].notnull()]['AJ_130_Score']

mod = sm.OLS(outcome.astype(float), df_data_exog.astype(float))
res = mod.fit()
print(res.summary())

/opt/anaconda3/envs/deeplearning/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


                            OLS Regression Results                            
Dep. Variable:           AJ_130_Score   R-squared:                       0.050
Model:                            OLS   Adj. R-squared:                  0.050
Method:                 Least Squares   F-statistic:                     85.34
Date:                Fri, 25 Jun 2021   Prob (F-statistic):           7.65e-37
Time:                        01:14:21   Log-Likelihood:                -20216.
No. Observations:                3231   AIC:                         4.044e+04
Df Residuals:                    3228   BIC:                         4.046e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
AGE                5.2693      0.406     12.

In [32]:
df_data = df_male

df_data_exog = sm.add_constant(df_data[df_data['Volume_Score'].notnull()][feature_mask], prepend=False)
outcome = df_data[df_data['Volume_Score'].notnull()]['Volume_Score']

mod = sm.OLS(outcome.astype(float), df_data_exog.astype(float))
res = mod.fit()
print(res.summary())

/opt/anaconda3/envs/deeplearning/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


                            OLS Regression Results                            
Dep. Variable:           Volume_Score   R-squared:                       0.037
Model:                            OLS   Adj. R-squared:                  0.035
Method:                 Least Squares   F-statistic:                     27.38
Date:                Fri, 25 Jun 2021   Prob (F-statistic):           2.14e-12
Time:                        01:14:21   Log-Likelihood:                -9163.0
No. Observations:                1444   AIC:                         1.833e+04
Df Residuals:                    1441   BIC:                         1.835e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
AGE                5.0634      0.685      7.

In [33]:
df_data = df_male

df_data_exog = sm.add_constant(df_data[df_data['SBP'].notnull()][feature_mask], prepend=False)
outcome = df_data[df_data['SBP'].notnull()]['SBP']

mod = sm.OLS(outcome.astype(float), df_data_exog.astype(float))
res = mod.fit()
print(res.summary())

/opt/anaconda3/envs/deeplearning/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


                            OLS Regression Results                            
Dep. Variable:                    SBP   R-squared:                       0.040
Model:                            OLS   Adj. R-squared:                  0.039
Method:                 Least Squares   F-statistic:                     189.6
Date:                Fri, 25 Jun 2021   Prob (F-statistic):           1.97e-81
Time:                        01:14:21   Log-Likelihood:                -35920.
No. Observations:                9193   AIC:                         7.185e+04
Df Residuals:                    9190   BIC:                         7.187e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
AGE               -0.1642      0.018     -9.

In [34]:
df_data = df_male

df_data_exog = sm.add_constant(df_data[df_data['DBP'].notnull()][feature_mask], prepend=False)
outcome = df_data[df_data['DBP'].notnull()]['DBP']

mod = sm.OLS(outcome.astype(float), df_data_exog.astype(float))
res = mod.fit()
print(res.summary())

/opt/anaconda3/envs/deeplearning/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


                            OLS Regression Results                            
Dep. Variable:                    DBP   R-squared:                       0.030
Model:                            OLS   Adj. R-squared:                  0.030
Method:                 Least Squares   F-statistic:                     143.6
Date:                Fri, 25 Jun 2021   Prob (F-statistic):           3.76e-62
Time:                        01:14:21   Log-Likelihood:                -32493.
No. Observations:                9193   AIC:                         6.499e+04
Df Residuals:                    9190   BIC:                         6.501e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
AGE               -0.0110      0.012     -0.

In [35]:
df_data = df_male

df_data_exog = sm.add_constant(df_data[df_data['Pulse_Pressure'].notnull()][feature_mask], prepend=False)
outcome = df_data[df_data['Pulse_Pressure'].notnull()]['Pulse_Pressure']

mod = sm.OLS(outcome.astype(float), df_data_exog.astype(float))
res = mod.fit()
print(res.summary())

/opt/anaconda3/envs/deeplearning/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


                            OLS Regression Results                            
Dep. Variable:         Pulse_Pressure   R-squared:                       0.020
Model:                            OLS   Adj. R-squared:                  0.020
Method:                 Least Squares   F-statistic:                     94.10
Date:                Fri, 25 Jun 2021   Prob (F-statistic):           3.50e-41
Time:                        01:14:21   Log-Likelihood:                -33247.
No. Observations:                9193   AIC:                         6.650e+04
Df Residuals:                    9190   BIC:                         6.652e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
AGE               -0.1532      0.013    -11.

#### Female

In [36]:
df_data = df_female

df_data_exog = sm.add_constant(df_data[df_data['mean_baPWV'].notnull()][feature_mask], prepend=False)
outcome = df_data[df_data['mean_baPWV'].notnull()]['mean_baPWV']

mod = sm.OLS(outcome.astype(float), df_data_exog.astype(float))
res = mod.fit()
print(res.summary())
print("\n")


df_data_exog = sm.add_constant(df_data[df_data['mean_ABI'].notnull()][feature_mask], prepend=False)
outcome = df_data[df_data['mean_ABI'].notnull()]['mean_ABI']

mod = sm.OLS(outcome.astype(float), df_data_exog.astype(float))
res = mod.fit()
print(res.summary())
print("\n")

df_data_exog = sm.add_constant(df_data[df_data['mean_IMT'].notnull()][feature_mask], prepend=False)
outcome = df_data[df_data['mean_IMT'].notnull()]['mean_IMT']

mod = sm.OLS(outcome.astype(float), df_data_exog.astype(float))
res = mod.fit()
print(res.summary())
print("\n")

df_data_exog = sm.add_constant(df_data[df_data['AJ_130_Score'].notnull()][feature_mask], prepend=False)
outcome = df_data[df_data['AJ_130_Score'].notnull()]['AJ_130_Score']

mod = sm.OLS(outcome.astype(float), df_data_exog.astype(float))
res = mod.fit()
print(res.summary())
print("\n")

df_data_exog = sm.add_constant(df_data[df_data['Volume_Score'].notnull()][feature_mask], prepend=False)
outcome = df_data[df_data['Volume_Score'].notnull()]['Volume_Score']

mod = sm.OLS(outcome.astype(float), df_data_exog.astype(float))
res = mod.fit()
print(res.summary())
print("\n")

df_data_exog = sm.add_constant(df_data[df_data['SBP'].notnull()][feature_mask], prepend=False)
outcome = df_data[df_data['SBP'].notnull()]['SBP']

mod = sm.OLS(outcome.astype(float), df_data_exog.astype(float))
res = mod.fit()
print(res.summary())
print("\n")

df_data_exog = sm.add_constant(df_data[df_data['DBP'].notnull()][feature_mask], prepend=False)
outcome = df_data[df_data['DBP'].notnull()]['DBP']

mod = sm.OLS(outcome.astype(float), df_data_exog.astype(float))
res = mod.fit()
print(res.summary())
print("\n")

df_data_exog = sm.add_constant(df_data[df_data['Pulse_Pressure'].notnull()][feature_mask], prepend=False)
outcome = df_data[df_data['Pulse_Pressure'].notnull()]['Pulse_Pressure']

mod = sm.OLS(outcome.astype(float), df_data_exog.astype(float))
res = mod.fit()
print(res.summary())
print("\n")

/opt/anaconda3/envs/deeplearning/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


                            OLS Regression Results                            
Dep. Variable:             mean_baPWV   R-squared:                       0.145
Model:                            OLS   Adj. R-squared:                  0.143
Method:                 Least Squares   F-statistic:                     70.94
Date:                Fri, 25 Jun 2021   Prob (F-statistic):           3.50e-29
Time:                        01:14:21   Log-Likelihood:                -5373.8
No. Observations:                 838   AIC:                         1.075e+04
Df Residuals:                     835   BIC:                         1.077e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
AGE               11.2434      1.048     10.

                            OLS Regression Results                            
Dep. Variable:                    SBP   R-squared:                       0.058
Model:                            OLS   Adj. R-squared:                  0.057
Method:                 Least Squares   F-statistic:                     47.87
Date:                Fri, 25 Jun 2021   Prob (F-statistic):           6.65e-21
Time:                        01:14:21   Log-Likelihood:                -6090.1
No. Observations:                1560   AIC:                         1.219e+04
Df Residuals:                    1557   BIC:                         1.220e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
AGE                0.1688      0.053      3.

### Model 3

In [37]:
feature_mask = ['AGE', 'Age^2', 'percentage_fat', 'CRF'] # Feature
feature_mask = ['AGE', 'percentage_fat', 'CRF']

/opt/anaconda3/envs/deeplearning/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


#### Male

In [38]:
df_data = df_male

df_data_exog = sm.add_constant(df_data[df_data['mean_baPWV'].notnull()][feature_mask], prepend=False)
outcome = df_data[df_data['mean_baPWV'].notnull()]['mean_baPWV']

mod = sm.OLS(outcome.astype(float), df_data_exog.astype(float))
res = mod.fit()
print(res.summary())
print("\n")


df_data_exog = sm.add_constant(df_data[df_data['mean_ABI'].notnull()][feature_mask], prepend=False)
outcome = df_data[df_data['mean_ABI'].notnull()]['mean_ABI']

mod = sm.OLS(outcome.astype(float), df_data_exog.astype(float))
res = mod.fit()
print(res.summary())
print("\n")

df_data_exog = sm.add_constant(df_data[df_data['mean_IMT'].notnull()][feature_mask], prepend=False)
outcome = df_data[df_data['mean_IMT'].notnull()]['mean_IMT']

mod = sm.OLS(outcome.astype(float), df_data_exog.astype(float))
res = mod.fit()
print(res.summary())
print("\n")

df_data_exog = sm.add_constant(df_data[df_data['AJ_130_Score'].notnull()][feature_mask], prepend=False)
outcome = df_data[df_data['AJ_130_Score'].notnull()]['AJ_130_Score']

mod = sm.OLS(outcome.astype(float), df_data_exog.astype(float))
res = mod.fit()
print(res.summary())
print("\n")

df_data_exog = sm.add_constant(df_data[df_data['Volume_Score'].notnull()][feature_mask], prepend=False)
outcome = df_data[df_data['Volume_Score'].notnull()]['Volume_Score']

mod = sm.OLS(outcome.astype(float), df_data_exog.astype(float))
res = mod.fit()
print(res.summary())
print("\n")

df_data_exog = sm.add_constant(df_data[df_data['SBP'].notnull()][feature_mask], prepend=False)
outcome = df_data[df_data['SBP'].notnull()]['SBP']

mod = sm.OLS(outcome.astype(float), df_data_exog.astype(float))
res = mod.fit()
print(res.summary())
print("\n")

df_data_exog = sm.add_constant(df_data[df_data['DBP'].notnull()][feature_mask], prepend=False)
outcome = df_data[df_data['DBP'].notnull()]['DBP']

mod = sm.OLS(outcome.astype(float), df_data_exog.astype(float))
res = mod.fit()
print(res.summary())
print("\n")

df_data_exog = sm.add_constant(df_data[df_data['Pulse_Pressure'].notnull()][feature_mask], prepend=False)
outcome = df_data[df_data['Pulse_Pressure'].notnull()]['Pulse_Pressure']

mod = sm.OLS(outcome.astype(float), df_data_exog.astype(float))
res = mod.fit()
print(res.summary())
print("\n")

/opt/anaconda3/envs/deeplearning/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


                            OLS Regression Results                            
Dep. Variable:             mean_baPWV   R-squared:                       0.066
Model:                            OLS   Adj. R-squared:                  0.065
Method:                 Least Squares   F-statistic:                     73.23
Date:                Fri, 25 Jun 2021   Prob (F-statistic):           9.19e-46
Time:                        01:14:21   Log-Likelihood:                -20429.
No. Observations:                3119   AIC:                         4.087e+04
Df Residuals:                    3115   BIC:                         4.089e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
AGE                5.5722      0.509     10.

                            OLS Regression Results                            
Dep. Variable:           Volume_Score   R-squared:                       0.043
Model:                            OLS   Adj. R-squared:                  0.041
Method:                 Least Squares   F-statistic:                     21.33
Date:                Fri, 25 Jun 2021   Prob (F-statistic):           1.61e-13
Time:                        01:14:21   Log-Likelihood:                -9158.5
No. Observations:                1444   AIC:                         1.833e+04
Df Residuals:                    1440   BIC:                         1.835e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
AGE                4.3908      0.719      6.

#### Female

In [39]:
df_data = df_female

df_data_exog = sm.add_constant(df_data[df_data['mean_baPWV'].notnull()][feature_mask], prepend=False)
outcome = df_data[df_data['mean_baPWV'].notnull()]['mean_baPWV']

mod = sm.OLS(outcome.astype(float), df_data_exog.astype(float))
res = mod.fit()
print(res.summary())
print("\n")


df_data_exog = sm.add_constant(df_data[df_data['mean_ABI'].notnull()][feature_mask], prepend=False)
outcome = df_data[df_data['mean_ABI'].notnull()]['mean_ABI']

mod = sm.OLS(outcome.astype(float), df_data_exog.astype(float))
res = mod.fit()
print(res.summary())
print("\n")

df_data_exog = sm.add_constant(df_data[df_data['mean_IMT'].notnull()][feature_mask], prepend=False)
outcome = df_data[df_data['mean_IMT'].notnull()]['mean_IMT']

mod = sm.OLS(outcome.astype(float), df_data_exog.astype(float))
res = mod.fit()
print(res.summary())
print("\n")

df_data_exog = sm.add_constant(df_data[df_data['AJ_130_Score'].notnull()][feature_mask], prepend=False)
outcome = df_data[df_data['AJ_130_Score'].notnull()]['AJ_130_Score']

mod = sm.OLS(outcome.astype(float), df_data_exog.astype(float))
res = mod.fit()
print(res.summary())
print("\n")

df_data_exog = sm.add_constant(df_data[df_data['Volume_Score'].notnull()][feature_mask], prepend=False)
outcome = df_data[df_data['Volume_Score'].notnull()]['Volume_Score']

mod = sm.OLS(outcome.astype(float), df_data_exog.astype(float))
res = mod.fit()
print(res.summary())
print("\n")

df_data_exog = sm.add_constant(df_data[df_data['SBP'].notnull()][feature_mask], prepend=False)
outcome = df_data[df_data['SBP'].notnull()]['SBP']

mod = sm.OLS(outcome.astype(float), df_data_exog.astype(float))
res = mod.fit()
print(res.summary())
print("\n")

df_data_exog = sm.add_constant(df_data[df_data['DBP'].notnull()][feature_mask], prepend=False)
outcome = df_data[df_data['DBP'].notnull()]['DBP']

mod = sm.OLS(outcome.astype(float), df_data_exog.astype(float))
res = mod.fit()
print(res.summary())
print("\n")

df_data_exog = sm.add_constant(df_data[df_data['Pulse_Pressure'].notnull()][feature_mask], prepend=False)
outcome = df_data[df_data['Pulse_Pressure'].notnull()]['Pulse_Pressure']

mod = sm.OLS(outcome.astype(float), df_data_exog.astype(float))
res = mod.fit()
print(res.summary())
print("\n")

                            OLS Regression Results                            
Dep. Variable:             mean_baPWV   R-squared:                       0.145
Model:                            OLS   Adj. R-squared:                  0.142
Method:                 Least Squares   F-statistic:                     47.28
Date:                Fri, 25 Jun 2021   Prob (F-statistic):           3.17e-28
Time:                        01:14:22   Log-Likelihood:                -5373.8
No. Observations:                 838   AIC:                         1.076e+04
Df Residuals:                     834   BIC:                         1.077e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
AGE               11.1818      1.064     10.

                            OLS Regression Results                            
Dep. Variable:                    SBP   R-squared:                       0.058
Model:                            OLS   Adj. R-squared:                  0.056
Method:                 Least Squares   F-statistic:                     31.90
Date:                Fri, 25 Jun 2021   Prob (F-statistic):           5.20e-20
Time:                        01:14:22   Log-Likelihood:                -6090.1
No. Observations:                1560   AIC:                         1.219e+04
Df Residuals:                    1556   BIC:                         1.221e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
AGE                0.1676      0.054      3.

/opt/anaconda3/envs/deeplearning/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Model 4

In [40]:
feature_mask = ['AGE', 'Age^2', 'percentage_fat', 'CRF', 'rest_HR', 'Glucose, Fasting', 
                'TG', 'CHOLESTEROL', 'Smoke', 'ALC']

feature_mask = ['AGE', 'percentage_fat', 'CRF', 'rest_HR', 'Glucose, Fasting', 
                'TG', 'CHOLESTEROL', 'Smoke', 'ALC']

/opt/anaconda3/envs/deeplearning/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


#### Male

In [41]:
df_data = df_male

df_data_exog = sm.add_constant(df_data[df_data['mean_baPWV'].notnull()][feature_mask].fillna(df_data[df_data['mean_baPWV'].notnull()].median()), prepend=False)
outcome = df_data[df_data['mean_baPWV'].notnull()]['mean_baPWV']

mod = sm.OLS(outcome.astype(float), df_data_exog.astype(float))
res = mod.fit()
print(res.summary())
print("\n")


df_data_exog = sm.add_constant(df_data[df_data['mean_ABI'].notnull()][feature_mask].fillna(df_data[df_data['mean_ABI'].notnull()].median()), prepend=False)
outcome = df_data[df_data['mean_ABI'].notnull()]['mean_ABI']

mod = sm.OLS(outcome.astype(float), df_data_exog.astype(float))
res = mod.fit()
print(res.summary())
print("\n")

df_data_exog = sm.add_constant(df_data[df_data['mean_IMT'].notnull()][feature_mask].fillna(df_data[df_data['mean_IMT'].notnull()].median()), prepend=False)
outcome = df_data[df_data['mean_IMT'].notnull()]['mean_IMT']

mod = sm.OLS(outcome.astype(float), df_data_exog.astype(float))
res = mod.fit()
print(res.summary())
print("\n")

df_data_exog = sm.add_constant(df_data[df_data['AJ_130_Score'].notnull()][feature_mask].fillna(df_data[df_data['AJ_130_Score'].notnull()].median()), prepend=False)
outcome = df_data[df_data['AJ_130_Score'].notnull()]['AJ_130_Score']

mod = sm.OLS(outcome.astype(float), df_data_exog.astype(float))
res = mod.fit()
print(res.summary())
print("\n")

df_data_exog = sm.add_constant(df_data[df_data['Volume_Score'].notnull()][feature_mask].fillna(df_data[df_data['Volume_Score'].notnull()].median()), prepend=False)
outcome = df_data[df_data['Volume_Score'].notnull()]['Volume_Score']

mod = sm.OLS(outcome.astype(float), df_data_exog.astype(float))
res = mod.fit()
print(res.summary())
print("\n")

df_data_exog = sm.add_constant(df_data[df_data['SBP'].notnull()][feature_mask].fillna(df_data[df_data['SBP'].notnull()].median()), prepend=False)
outcome = df_data[df_data['SBP'].notnull()]['SBP']

mod = sm.OLS(outcome.astype(float), df_data_exog.astype(float))
res = mod.fit()
print(res.summary())
print("\n")

df_data_exog = sm.add_constant(df_data[df_data['DBP'].notnull()][feature_mask].fillna(df_data[df_data['DBP'].notnull()].median()), prepend=False)
outcome = df_data[df_data['DBP'].notnull()]['DBP']

mod = sm.OLS(outcome.astype(float), df_data_exog.astype(float))
res = mod.fit()
print(res.summary())
print("\n")

df_data_exog = sm.add_constant(df_data[df_data['Pulse_Pressure'].notnull()][feature_mask].fillna(df_data[df_data['Pulse_Pressure'].notnull()].median()), prepend=False)
outcome = df_data[df_data['Pulse_Pressure'].notnull()]['Pulse_Pressure']

mod = sm.OLS(outcome.astype(float), df_data_exog.astype(float))
res = mod.fit()
print(res.summary())
print("\n")

/opt/anaconda3/envs/deeplearning/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


                            OLS Regression Results                            
Dep. Variable:             mean_baPWV   R-squared:                       0.113
Model:                            OLS   Adj. R-squared:                  0.111
Method:                 Least Squares   F-statistic:                     44.08
Date:                Fri, 25 Jun 2021   Prob (F-statistic):           5.37e-75
Time:                        01:14:22   Log-Likelihood:                -20348.
No. Observations:                3119   AIC:                         4.072e+04
Df Residuals:                    3109   BIC:                         4.078e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
AGE                  5.9957      0.519  

                            OLS Regression Results                            
Dep. Variable:           Volume_Score   R-squared:                       0.081
Model:                            OLS   Adj. R-squared:                  0.075
Method:                 Least Squares   F-statistic:                     13.99
Date:                Fri, 25 Jun 2021   Prob (F-statistic):           8.30e-22
Time:                        01:14:22   Log-Likelihood:                -9129.1
No. Observations:                1444   AIC:                         1.828e+04
Df Residuals:                    1434   BIC:                         1.833e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
AGE                  4.1569      0.743  

#### Female

In [42]:
df_data = df_female

df_data_exog = sm.add_constant(df_data[df_data['mean_baPWV'].notnull()][feature_mask].fillna(df_data[df_data['mean_baPWV'].notnull()].median()), prepend=False)
outcome = df_data[df_data['mean_baPWV'].notnull()]['mean_baPWV']

mod = sm.OLS(outcome.astype(float), df_data_exog.astype(float))
res = mod.fit()
print(res.summary())
print("\n")


df_data_exog = sm.add_constant(df_data[df_data['mean_ABI'].notnull()][feature_mask].fillna(df_data[df_data['mean_ABI'].notnull()].median()), prepend=False)
outcome = df_data[df_data['mean_ABI'].notnull()]['mean_ABI']

mod = sm.OLS(outcome.astype(float), df_data_exog.astype(float))
res = mod.fit()
print(res.summary())
print("\n")

df_data_exog = sm.add_constant(df_data[df_data['mean_IMT'].notnull()][feature_mask].fillna(df_data[df_data['mean_IMT'].notnull()].median()), prepend=False)
outcome = df_data[df_data['mean_IMT'].notnull()]['mean_IMT']

mod = sm.OLS(outcome.astype(float), df_data_exog.astype(float))
res = mod.fit()
print(res.summary())
print("\n")

df_data_exog = sm.add_constant(df_data[df_data['AJ_130_Score'].notnull()][feature_mask].fillna(df_data[df_data['AJ_130_Score'].notnull()].median()), prepend=False)
outcome = df_data[df_data['AJ_130_Score'].notnull()]['AJ_130_Score']

mod = sm.OLS(outcome.astype(float), df_data_exog.astype(float))
res = mod.fit()
print(res.summary())
print("\n")

df_data_exog = sm.add_constant(df_data[df_data['Volume_Score'].notnull()][feature_mask].fillna(df_data[df_data['Volume_Score'].notnull()].median()), prepend=False)
outcome = df_data[df_data['Volume_Score'].notnull()]['Volume_Score']

mod = sm.OLS(outcome.astype(float), df_data_exog.astype(float))
res = mod.fit()
print(res.summary())
print("\n")

df_data_exog = sm.add_constant(df_data[df_data['SBP'].notnull()][feature_mask].fillna(df_data[df_data['SBP'].notnull()].median()), prepend=False)
outcome = df_data[df_data['SBP'].notnull()]['SBP']

mod = sm.OLS(outcome.astype(float), df_data_exog.astype(float))
res = mod.fit()
print(res.summary())
print("\n")

df_data_exog = sm.add_constant(df_data[df_data['DBP'].notnull()][feature_mask].fillna(df_data[df_data['DBP'].notnull()].median()), prepend=False)
outcome = df_data[df_data['DBP'].notnull()]['DBP']

mod = sm.OLS(outcome.astype(float), df_data_exog.astype(float))
res = mod.fit()
print(res.summary())
print("\n")

df_data_exog = sm.add_constant(df_data[df_data['Pulse_Pressure'].notnull()][feature_mask].fillna(df_data[df_data['Pulse_Pressure'].notnull()].median()), prepend=False)
outcome = df_data[df_data['Pulse_Pressure'].notnull()]['Pulse_Pressure']

mod = sm.OLS(outcome.astype(float), df_data_exog.astype(float))
res = mod.fit()
print(res.summary())
print("\n")

                            OLS Regression Results                            
Dep. Variable:             mean_baPWV   R-squared:                       0.198
Model:                            OLS   Adj. R-squared:                  0.189
Method:                 Least Squares   F-statistic:                     22.69
Date:                Fri, 25 Jun 2021   Prob (F-statistic):           1.03e-34
Time:                        01:14:22   Log-Likelihood:                -5347.2
No. Observations:                 838   AIC:                         1.071e+04
Df Residuals:                     828   BIC:                         1.076e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
AGE                 11.9766      1.089  

                            OLS Regression Results                            
Dep. Variable:                    DBP   R-squared:                       0.071
Model:                            OLS   Adj. R-squared:                  0.066
Method:                 Least Squares   F-statistic:                     13.23
Date:                Fri, 25 Jun 2021   Prob (F-statistic):           1.44e-20
Time:                        01:14:22   Log-Likelihood:                -5658.4
No. Observations:                1560   AIC:                         1.134e+04
Df Residuals:                    1550   BIC:                         1.139e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
AGE                  0.0417      0.043  

/opt/anaconda3/envs/deeplearning/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
